In [1]:
 # model summary
from torchinfo import summary
from torch.utils.data import DataLoader
from transformers import CamembertForMaskedLM, CamembertTokenizer, TrainingArguments, AdamW, Trainer , CamembertConfig
from datasets import load_from_disk
from dataset import OscarDataset
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
# === Model === #
#config = CamembertConfig(
 #    vocab_size=tokenizer.vocab_size)  # Adjust to match your tokenizer's vocab size
#     hidden_size=768,                 # Hidden size (RoBERTa_BASE)
#     num_hidden_layers=12,            # Number of transformer layers
#     num_attention_heads=12,          # Number of attention heads
#     intermediate_size=3072,          # FFN inner hidden size
#     hidden_dropout_prob=0.1,         # Dropout probability
#     attention_probs_dropout_prob=0.1, # Attention dropout probability
#     max_position_embeddings=514,     # Maximum sequence length + special tokens
#     type_vocab_size=1,               # No token type embeddings
#     initializer_range=0.02           # Standard deviation for weight initialization
# )

config = CamembertConfig()
print(config)
# Initialize a randomly weighted CamembertForMaskedLM model
model = CamembertForMaskedLM(config) 
#model.to("cuda")

print("Model initialized")
print(model)
summary(model)



c:\Users\salhi\.conda\envs\nlp\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


c:\Users\salhi\.conda\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CamembertConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "camembert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Model initialized
CamembertForMaskedLM(
  (roberta): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder):

Layer (type:depth-idx)                                       Param #
CamembertForMaskedLM                                         --
├─CamembertModel: 1-1                                        --
│    └─CamembertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                                   23,440,896
│    │    └─Embedding: 3-2                                   393,216
│    │    └─Embedding: 3-3                                   1,536
│    │    └─LayerNorm: 3-4                                   1,536
│    │    └─Dropout: 3-5                                     --
│    └─CamembertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                                  85,054,464
├─CamembertLMHead: 1-2                                       --
│    └─Linear: 2-3                                           590,592
│    └─LayerNorm: 2-4                                        1,536
│    └─Linear: 2-5                                           23,

In [ ]:
import torch
import torch.nn as nn

class CamembertEmbeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size, padding_idx=config.pad_token_id)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

        self.position_ids = torch.arange(config.max_position_embeddings).unsqueeze(0)  # Shape (1, max_position_embeddings)
        self.token_type_ids = torch.zeros_like(self.position_ids, dtype=torch.long)  # Shape (1, max_position_embeddings)

    def forward(self, input_ids, token_type_ids=None, position_ids=None):
        input_shape = input_ids.size()
        batch_size, seq_length = input_shape

        if position_ids is None:
            position_ids = self.position_ids[:, :seq_length].to(input_ids.device)
        if token_type_ids is None:
            token_type_ids = self.token_type_ids[:, :seq_length].expand(batch_size, seq_length).to(input_ids.device)

        inputs_embeds = self.word_embeddings(input_ids)
        position_embeds = self.position_embeddings(position_ids)
        token_type_embeds = self.token_type_embeddings(token_type_ids)
        embeddings = inputs_embeds + position_embeds + token_type_embeds

        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)

        return embeddings


In [31]:
from transformers import CamembertConfig
class Config:
    vocab_size = 30522  # Match this with the pretrained tokenizer's vocab size
    hidden_size = 768
    max_position_embeddings = 512
    type_vocab_size = 2
    hidden_dropout_prob = 0.1
    layer_norm_eps = 1e-12
    pad_token_id = 1


# Initialize configuration and embeddings
config = CamembertConfig()
embeddings = CamembertEmbeddings(config)

# Sample inputs
input_ids = torch.randint(0, config.vocab_size, (2, 10))  # Batch of 2, sequence length of 10
output = embeddings(input_ids)
summary(embeddings)  # Output: (batch_size, sequence_length, hidden_size)
  # Output: (batch_size, sequence_length, hidden_size)


Layer (type:depth-idx)                   Param #
CamembertEmbeddings                      --
├─Embedding: 1-1                         23,440,896
├─Embedding: 1-2                         393,216
├─Embedding: 1-3                         1,536
├─LayerNorm: 1-4                         1,536
├─Dropout: 1-5                           --
Total params: 23,837,184
Trainable params: 23,837,184
Non-trainable params: 0

In [ ]:
import torch
import torch.nn as nn
import math

class CamembertSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = config.hidden_size // config.num_attention_heads
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

    def transpose_for_scores(self, x):
        batch_size, seq_length, hidden_size = x.size()
        x = x.view(batch_size, seq_length, self.num_attention_heads, self.attention_head_size)
        return x.permute(0, 2, 1, 3)  # [batch, num_heads, seq_len, head_size]

    def forward(self, hidden_states, attention_mask=None):
        # Linear projections
        query_layer = self.transpose_for_scores(self.query(hidden_states))
        key_layer = self.transpose_for_scores(self.key(hidden_states))
        value_layer = self.transpose_for_scores(self.value(hidden_states))

        # Compute attention scores
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores /= math.sqrt(self.attention_head_size)

        # Apply attention mask (broadcast to match attention_scores shape)
        if attention_mask is not None:
            attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)  # [batch, 1, 1, seq_len]
            attention_scores += attention_mask

        # Normalize attention scores to probabilities
        attention_probs = nn.functional.softmax(attention_scores, dim=-1)
        attention_probs = self.dropout(attention_probs)

        # Weighted sum of value vectors
        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        context_layer = context_layer.view(hidden_states.size(0), -1, self.all_head_size)

        return context_layer

class CamembertFeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense_1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.activation = nn.ReLU()  # Simpler activation for beginners
        self.dense_2 = nn.Linear(config.intermediate_size, config.hidden_size)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, x):
        x = self.dense_1(x)
        x = self.activation(x)
        x = self.dense_2(x)
        return self.dropout(x)

class CamembertLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.attention = CamembertSelfAttention(config)
        self.attention_norm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.feed_forward = CamembertFeedForward(config)
        self.feed_forward_norm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)

    def forward(self, hidden_states, attention_mask=None):
        # Self-Attention with skip connection
        attention_output = self.attention(hidden_states, attention_mask)
        attention_output = self.attention_norm(hidden_states + attention_output)  # Skip connection

        # Feed-Forward with skip connection
        feed_forward_output = self.feed_forward(attention_output)
        layer_output = self.feed_forward_norm(attention_output + feed_forward_output)  # Skip connection

        return layer_output

class CamembertEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layers = nn.ModuleList([CamembertLayer(config) for _ in range(config.num_hidden_layers)])

    def forward(self, hidden_states, attention_mask=None):
        for layer in self.layers:
            hidden_states = layer(hidden_states, attention_mask)
        return hidden_states
    
class CamembertModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embeddings = CamembertEmbeddings(config)  # Ensure it includes word, position, and type embeddings
        self.encoder = CamembertEncoder(config)  # Ensure num_hidden_layers matches config

    def forward(self, input_ids, attention_mask=None):
        # Embedding layer
        embedded_input = self.embeddings(input_ids)

        # Attention mask preparation
        if attention_mask is not None:
            attention_mask = (1.0 - attention_mask) * -10000.0  # Convert to large negative values for masked positions

        # Encoder
        encoder_output = self.encoder(embedded_input, attention_mask)
        return encoder_output


In [33]:
# Define a config with essential parameters
class Config:
    hidden_size = 768
    num_attention_heads = 12
    intermediate_size = 3072
    hidden_dropout_prob = 0.1
    attention_probs_dropout_prob = 0.1
    num_hidden_layers = 12
    layer_norm_eps = 1e-12
    vocab_size = 30522
    max_position_embeddings = 512

config = CamembertConfig()

# Create model
model = CamembertModel(config)

# Sample inputs
input_ids = torch.randint(0, config.vocab_size, (2, 10))  # Batch size of 2, sequence length of 10
attention_mask = torch.ones(2, 10)  # No masking

# Forward pass
output = model(input_ids, attention_mask)
print(output.shape)  # Should be (2, 10, 768)
summary(model)  # Output: (batch_size, sequence_length, hidden_size)

torch.Size([2, 10, 768])


Layer (type:depth-idx)                             Param #
CamembertModel                                     --
├─CamembertEmbeddings: 1-1                         --
│    └─Embedding: 2-1                              23,440,896
│    └─Embedding: 2-2                              393,216
│    └─Embedding: 2-3                              1,536
│    └─LayerNorm: 2-4                              1,536
│    └─Dropout: 2-5                                --
├─CamembertEncoder: 1-2                            --
│    └─ModuleList: 2-6                             --
│    │    └─CamembertLayer: 3-1                    6,497,280
│    │    └─CamembertLayer: 3-2                    6,497,280
│    │    └─CamembertLayer: 3-3                    6,497,280
│    │    └─CamembertLayer: 3-4                    6,497,280
│    │    └─CamembertLayer: 3-5                    6,497,280
│    │    └─CamembertLayer: 3-6                    6,497,280
│    │    └─CamembertLayer: 3-7                    6,497,280
│    │   